In [1]:
from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.local import LocalQuantumJob
from braket.jobs.image_uris import Framework, retrieve_image
import time

aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")

In [2]:
num_total_updates = 500

hyperparams = {
    "num_total_updates": num_total_updates, 
}

#Name your job so that it can be later identified
job_name = f'cx-gate-calibration-{num_total_updates}-updates-{str(int(time.time()))}'

# Specify one of the ready-to-use containers provided by Amazon Braket
image_uri = retrieve_image(Framework.PL_PYTORCH, AwsSession().region)

q_env_config_path = 'config_yamls/q_env_gate_config.yaml'
agent_config_path = 'config_yamls/agent_config.yaml'

input_config = {
    'q-env-config': q_env_config_path,
    'agent-config': agent_config_path,
}

In [3]:
job = AwsQuantumJob.create(
    #Run this hybrid job on the SV1 simulator
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    #The directory or single file containing the code to run.
    source_module="needed_files",
    #The main script or function the job will run.
    entry_point="needed_files.algorithm_script:calibrate_gate",
    #Set the name of the job
    job_name=job_name,
    # Set the hyperparameters
    hyperparameters=hyperparams,
    # Specify the image to use for the container to run the script in
    # image_uri=image_uri,
    # Define the file that contains the input data but not relevant for the minimal example
    input_data=input_config,
    wait_until_complete=True,
    aws_session=aws_session,
)

Initializing Braket Job: arn:aws:braket:us-west-1:229406906664:job/cx-gate-calibration-500-updates-1706261416
.........................
2024-01-26 09:32:32,695 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-01-26 09:32:32,696 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-01-26 09:32:32,709 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-01-26 09:32:32,710 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-01-26 09:32:32,721 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-01-26 09:32:32,721 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-01-26 09:32:32,732 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "agent-config": "/opt/ml/inp

In [4]:
while job.state() not in job.TERMINAL_STATES:
    print(job.state())
    time.sleep(30)

print(job.state())
print(job.result())

COMPLETED
{'final_action_vector': [-0.008470225147902966, -2.469658613204956, -2.719196319580078, 1.5587186813354492, -1.5609763860702515, 1.520658254623413, -1.6004196405410767]}
